In [60]:
import re
import ast
import numpy as np
import math
from functools import cache
from itertools import product

def read_file_to_string(file_path):
    with open(file_path,'r') as f:
        return f.read()
    
day_20_string = read_file_to_string('day_20_input.txt')
day_20_test = read_file_to_string('day_20_test.txt')
day_20_test_2 = read_file_to_string('day_20_test_2.txt')
# day_20_test_3 = read_file_to_string('day_20_test_3.txt')
# day_20_test_4 = read_file_to_string('day_20_test_4.txt')
# day_20_test_5 = read_file_to_string('day_20_test_5.txt')

In [66]:
def parse_string(string):
    day_20 = string.split('\n')

    string_dict = {}
    for string in day_20:
        if string[0] in '%&':
            slice = string[1:string.index(' ')]
            string_dict[slice] = {'symbol':string[0],'symb_on':0,'destination':[string_2.strip() for string_2 in re.split(',',string[string.rindex('>')+1:])]}

        else:
            slice = string[:string.index(' ')]
            string_dict[slice] = {'symbol':'','symb_on':0,'destination':[string_2.strip() for string_2 in re.split(',',string[string.rindex('>')+1:])]}


    
    return string_dict

In [36]:
test_rules = parse_string(day_20_test)

In [17]:
test_rules

{'broadcaster': {'symbol': '', 'symb_on': 0, 'destination': ['a', 'b', 'c']},
 'a': {'symbol': '%', 'symb_on': 0, 'destination': ['b']},
 'b': {'symbol': '%', 'symb_on': 0, 'destination': ['c']},
 'c': {'symbol': '%', 'symb_on': 0, 'destination': ['inv']},
 'inv': {'symbol': '&', 'symb_on': 0, 'destination': ['a']}}

In [77]:
def process(symb_dict, cycles = 1,start='broadcaster'):

    #use a stack, last in first out!

    #for &s
    remember_pulse_dict = {}

    for k,v in symb_dict.items():

        if v['symbol'] == '&':
            remember_pulse_dict[k] = {}


    total_low, total_high = 0,0
    #find a way to put things that go to that destination into the dict above
    for k,v in symb_dict.items():
        for k2 in remember_pulse_dict.keys():
            if k2 in v['destination']:
                remember_pulse_dict[k2][k] = 0

    for i in range(cycles):
        queue = [(start, 0, '')]
        low_pulses = 1
        high_pulses = 0
        while len(queue) > 0:

            location, pulse, came_from = queue.pop(0)
            #print(location,pulse)
            
            #check for whether our location is in these rules
            if location not in symb_dict.keys():
                continue

            #count what the pulse is
            if location == 'broadcaster':
                
                if pulse == 0:
                    #print("hello0")
                    low_pulses += len(symb_dict[location]['destination'])
                else:
                    #print("hello1")
                    high_pulses += len(symb_dict[location]['destination'])

                new_pulse = pulse


            #check if high pulse dies
            if symb_dict[location]['symbol'] == '%' and pulse == 1:
                continue
            
            #% when we have low pulse come in
            elif symb_dict[location]['symbol'] == '%' and pulse == 0:
                symb_dict[location]['symb_on'] = 1-symb_dict[location]['symb_on']
                new_pulse = symb_dict[location]['symb_on']
                
            
                if new_pulse == 0:
                    #print("hello0")
                    low_pulses += len(symb_dict[location]['destination'])
                else:
                    #print("hello1")
                    high_pulses += len(symb_dict[location]['destination'])

            #& when we have to bloody remember what happened
            elif symb_dict[location]['symbol'] == '&':

                remember_pulse_dict[location][came_from] = pulse

                if set(list(remember_pulse_dict[location].values())) == {1}:
                    new_pulse = 0
                else:
                    new_pulse = 1

            
          
                if new_pulse == 0:
                    #print("hello0")
                    low_pulses += len(symb_dict[location]['destination'])
                else:
                    #print("hello1")
                    high_pulses += len(symb_dict[location]['destination'])
            #need & logic to remember where things came from



        
            #append the stack to get the new locations, backwards to preserve first in/first out
            for new_loc in symb_dict[location]['destination']:
                queue.append((new_loc, new_pulse, location))
        
        total_high += high_pulses
        total_low += low_pulses

    return total_low,total_high

In [78]:
#should be 8,4
test_rules = parse_string(day_20_test)
process(test_rules, cycles=1,start='broadcaster')

(8, 4)

In [83]:
#should be 4,4
test_rules_2 = parse_string(day_20_test_2)
process(test_rules_2, cycles=1000,start='broadcaster')

(4250, 2750)

In [84]:
input_rules = parse_string(day_20_string)
process(input_rules, cycles=1000,start='broadcaster')

(18361, 44384)

In [85]:
18361*44384

814934624

In [95]:
def first_low_rx(symb_dict,start='broadcaster'):

    #use a stack, last in first out!

    #for &s
    remember_pulse_dict = {}

    for k,v in symb_dict.items():

        if v['symbol'] == '&':
            remember_pulse_dict[k] = {}


    total_low, total_high = 0,0
    #find a way to put things that go to that destination into the dict above
    for k,v in symb_dict.items():
        for k2 in remember_pulse_dict.keys():
            if k2 in v['destination']:
                remember_pulse_dict[k2][k] = 0

    i=1
    while True:
        #print(i)
        queue = [(start, 0, '')]
        low_pulses = 1
        high_pulses = 0
        while len(queue) > 0:

            location, pulse, came_from = queue.pop(0)
            #print(location,pulse)
            
            #check for whether our location is in these rules
            if location not in symb_dict.keys():
                continue

            #count what the pulse is
            if location == 'broadcaster':
                
                if pulse == 0:
                    #print("hello0")
                    low_pulses += len(symb_dict[location]['destination'])
                else:
                    #print("hello1")
                    high_pulses += len(symb_dict[location]['destination'])

                new_pulse = pulse


            #check if high pulse dies
            if symb_dict[location]['symbol'] == '%' and pulse == 1:
                continue
            
            #% when we have low pulse come in
            elif symb_dict[location]['symbol'] == '%' and pulse == 0:
                symb_dict[location]['symb_on'] = 1-symb_dict[location]['symb_on']
                new_pulse = symb_dict[location]['symb_on']
                
            
                if new_pulse == 0:
                    #print("hello0")
                    low_pulses += len(symb_dict[location]['destination'])
                else:
                    #print("hello1")
                    high_pulses += len(symb_dict[location]['destination'])

            #& when we have to bloody remember what happened
            elif symb_dict[location]['symbol'] == '&':

                remember_pulse_dict[location][came_from] = pulse

                if set(list(remember_pulse_dict[location].values())) == {1}:
                    if location == 'nr':
                        return i
                    new_pulse = 0
                else:
                    if location == 'nr':
                        if 1 in set(list(remember_pulse_dict[location].values())):
                            print(i,remember_pulse_dict[location])
                    new_pulse = 1
          
                if new_pulse == 0:
                    #print("hello0")
                    low_pulses += len(symb_dict[location]['destination'])
                else:
                    #print("hello1")
                    high_pulses += len(symb_dict[location]['destination'])
            #need & logic to remember where things came from
        
            #append the stack to get the new locations, backwards to preserve first in/first out
            for new_loc in symb_dict[location]['destination']:
                queue.append((new_loc, new_pulse, location))
        i+=1

In [101]:
input_rules = parse_string(day_20_string)
first_low_rx(input_rules)

3761 {'lh': 1, 'fk': 0, 'ff': 0, 'mm': 0}
3761 {'lh': 1, 'fk': 0, 'ff': 0, 'mm': 0}
3761 {'lh': 1, 'fk': 0, 'ff': 0, 'mm': 0}
3761 {'lh': 1, 'fk': 0, 'ff': 0, 'mm': 0}
3797 {'lh': 0, 'fk': 0, 'ff': 1, 'mm': 0}
3797 {'lh': 0, 'fk': 0, 'ff': 1, 'mm': 0}
3919 {'lh': 0, 'fk': 0, 'ff': 0, 'mm': 1}
3919 {'lh': 0, 'fk': 0, 'ff': 0, 'mm': 1}
3919 {'lh': 0, 'fk': 0, 'ff': 0, 'mm': 1}
4079 {'lh': 0, 'fk': 1, 'ff': 0, 'mm': 0}
4079 {'lh': 0, 'fk': 1, 'ff': 0, 'mm': 0}
4079 {'lh': 0, 'fk': 1, 'ff': 0, 'mm': 0}
7522 {'lh': 1, 'fk': 0, 'ff': 0, 'mm': 0}
7522 {'lh': 1, 'fk': 0, 'ff': 0, 'mm': 0}
7522 {'lh': 1, 'fk': 0, 'ff': 0, 'mm': 0}
7522 {'lh': 1, 'fk': 0, 'ff': 0, 'mm': 0}
7594 {'lh': 0, 'fk': 0, 'ff': 1, 'mm': 0}
7594 {'lh': 0, 'fk': 0, 'ff': 1, 'mm': 0}
7838 {'lh': 0, 'fk': 0, 'ff': 0, 'mm': 1}
7838 {'lh': 0, 'fk': 0, 'ff': 0, 'mm': 1}
7838 {'lh': 0, 'fk': 0, 'ff': 0, 'mm': 1}
8158 {'lh': 0, 'fk': 1, 'ff': 0, 'mm': 0}
8158 {'lh': 0, 'fk': 1, 'ff': 0, 'mm': 0}
8158 {'lh': 0, 'fk': 1, 'ff': 0, '

KeyboardInterrupt: 

In [102]:
math.lcm(3761,4079,3797,3919)

228282646835717

In [100]:
print(10631-6870)
6870-3109

3761


3761

87305194161900